# ETL Process Labor Market 

##### The purpose of this python script is to scrape web data from multiple websites all related to job postings, salaries and other important data points regarding the labor market in tech. The data collected from this notebook will then be loaded into CSV files which will be analyzed and cleaned in another script.

In [1]:
#neccessary libraries for webscraping + data ETL
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### This function extracts data from a website and parses it into HTML

In [2]:
#extracting the webpage that we want to parse
def extract(page1, page2=None):
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari"}
    url = f"{page1}={page2}"
    if page2 == None:
        url = f"{page1}"
    r = requests.get(url, headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

### The functions 'transform_2'  & 'parse_indeed' are used to take our unparsed html and convert it into a role variable that contains multiple data points based on a job posting

In [3]:
#transofmring our raw html data into a readable format
def transform_2(soup, destination):
    div = soup.find_all('div', class_ = 'slider_container')
    for i in div: 
        title = i.find('div', class_= 'heading4 color-text-primary singleLineTitle tapItem-gutter').text.strip()
        name = i.find('span', class_ = "companyName").text.strip()
        location = i.find('div', class_ = "heading6 company_location tapItem-gutter companyInfo").text.strip()
        rating = i.find('span', class_ = "ratingNumber").text.strip()
        summary = i.find('div', class_= 'job-snippet').text.strip()
        
        #loading our new variables into a dictionary
        role = {
            'Title': title,
            'Company_Name': name,
            'Location': location,
            'Rating': rating,
            'Summary': summary
        } 
        destination.append(role)

#this function parses data from indeed and places it inbto a df        
def parse_indeed(company, destination, search1=None, search2=None):
        c = extract(f"https://www.indeed.com/jobs?q={search1}%20{search2}%20{company}&vjk=877ae4112485d7c7&start", 0)
        for i in range(0, 100, 10):
            x = transform_2(c, destination)

In [45]:
#loading our nationwide data into a pandas df
nationwide_data_analytics = []
parse_indeed("Nationwide", nationwide_data_analytics, "Data", "Analytics")

In [43]:
nationwide_swe = []
parse_indeed("Nationwide", nationwide_swe, "Software", "Engineer")

In [49]:
nationwide_data_analytics = pd.DataFrame(nationwide_data_analytics)
nationwide_swe = pd.DataFrame(nationwide_swe)

In [11]:
#pulling data from wikipedia list of top 100 technology comapnies, for further data collection
r = requests.get("https://en.wikipedia.org/wiki/List_of_largest_Internet_companies")
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[1]
top_100 = df['Company']
top_100.at[1]='Google'
arr = np.array(top_100)

### These few cells loop through our array of the top_100 tech companies by revenue in order to search for various roles that being Analytics and SWE jobs 

In [38]:
#processes our parse_indeed function for each, name in the top_100 companies
all_frames = []

for i in arr:
    try:
        parse_indeed(i, all_frames, 'Data', 'Analytics')
    except AttributeError:
        print('Ignore')

Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore


In [51]:
all_swe = []
for i in arr:
    try:
        parse_indeed(i, all_swe, 'Software', 'Engineer')
    except AttributeError:
        print('Ignore')

Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore
Ignore


In [52]:
data_analytics_all = pd.DataFrame(all_frames)
swe_all = pd.DataFrame(all_swe)

In [57]:
#finishing our Indeed ETL Proces by loading our data into a CSV format for later analysis
"""
nationwide_data_analytics.to_csv('nationwide_data_analytics.csv')
nationwide_swe.to_csv('nationwide_swe.csv')
data_analytics_all.to_csv('data_analytics_all.csv')
swe_all.to_csv('swe_all.csv')
"""

# We will now switch our focus to collecting salary data from glassdoor and various other websites, related to SWE and Data Analytics Jobs

In [3]:
#first we must create two new functions that will process data scraped from glassdoor
def transform_salaries(soup, destination, title = None):
    #choosing which classes we want to extract data from 
    div = soup.find('div', class_ = 'css-14426yx eu4oa1w0')
    div2 = soup.find('div', class_ = 'css-1b8hwn9 eu4oa1w0')
    title = title
    
    #getting our data from the classes
    salary = soup.find('span', class_ = 'cmp-SalarySummaryAverage-salary cmp-SalarySummaryAverage-salary--summary css-mfbg43 e1wnkr790')
    company = soup.find('span', itemprop = 'name', class_ = 'css-tbayqg e1wnkr790')
    
    salary_data = { 'Title' : title,
                    'Salary' : salary,  
                    'Company': company
                  }
                   
    destination.append(salary_data)

def parse_salaries(company, destination, search1=None, search2=None, title = None):
        c = extract(f"https://www.indeed.com/cmp/{company}/salaries/{search1}-{search2}")
        x = transform_salaries(c, destination, title)
            

In [4]:
x = extract("https://www.indeed.com/cmp/Nationwide-Mutual-Insurance-Company/salaries/Data-Analyst")

In [5]:
test = []
transform_salaries(x, test)

In [6]:
#Extracting Nationwide Salary Data on Software Engineers, Data Analysts, Data Engineers, Data Scientists
nationwide_salary = []

parse_salaries('Nationwide Mutual Insurance Company', nationwide_salary, 'Data', 'Analyst', 'Data-Analyst')
parse_salaries('Nationwide Mutual Insurance Company', nationwide_salary, 'Data', 'Engineer', 'Data-Engineer')
parse_salaries('Nationwide Mutual Insurance Company', nationwide_salary, 'Data', 'Scientist', 'Data-Scientist')
parse_salaries('Nationwide Mutual Insurance Company', nationwide_salary, 'Software', 'Engineer', 'Software-Engineer')

In [7]:
nationwide_frame = pd.DataFrame(nationwide_salary)

### Here we are now using a new function to parse salary data from the top 100 companies in the tech space. We then save that data in various dataframes and save it to CSV files

In [8]:
def parse_top100_salarires(destination, search1, search2, title):
    for i in arr:
        try:
            parse_salaries(i, destination, search1, search2, title)
        except AttributeError:
            print('Ignore')

In [12]:
data_engineer_aggr = []
parse_top100_salarires(data_engineer_aggr, 'Data', 'Engineer', 'Data Engineer')

In [13]:
data_analyst_aggr = []
parse_top100_salarires(data_analyst_aggr, 'Data', 'Analyst', 'Data Analyst')

In [14]:
data_science_aggr = []
parse_top100_salarires(data_science_aggr, 'Data', 'Scientist', 'Data Scientist')

In [15]:
swe_aggr = []
parse_top100_salarires(swe_aggr, 'Software', 'Engineer', 'Software Engineer')

In [16]:
data_engineer_all = pd.DataFrame(data_engineer_aggr).dropna()
data_analyst_all = pd.DataFrame(data_analyst_aggr).dropna()
data_science_all = pd.DataFrame(data_science_aggr).dropna()
swe_all = pd.DataFrame(swe_aggr).dropna()

In [17]:
#converting our dataframes to CSV files
""""
nationwide_frame.to_csv('nationwide_sal.csv')
data_engineer_all.to_csv('data_engineer_sal.csv')
data_analyst_all.to_csv('data_analyst_sal.csv')
data_science_all.to_csv('data_science_sal.csv')
swe_all.to_csv('swe_sal.csv') 
""""